In [23]:
from pandas import Series
import pandas as pd
import numpy as np
import collections

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import recall_score, precision_score, f1_score, log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score

import matplotlib
plt.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 8)

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

from datetime import date
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

чтобы уменшить размер чтобы гитхаб принимает до 25мб пришлось коды как коммент сделать


In [24]:
import plotly.express as px

In [25]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [26]:
RANDOM_SEED = 42

In [1]:
#train.head()

In [2]:
#train.info()

In [3]:
#train.isnull().sum()

In [4]:
#test.head()

In [5]:
#test.info()

In [6]:
test.isnull().sum()

NameError: name 'test' is not defined

In [7]:
#sns.countplot(train['education'])

In [8]:
#sns.countplot(test['education'])

In [35]:
train = train.fillna('SCH')
test = test.fillna('SCH')

In [9]:
#train.isnull().sum(), test.isnull().sum()

In [10]:
#train.app_date

In [38]:
train['app_date'] = train['app_date'].apply(lambda x: datetime.strptime(x, '%d%b%Y'))

In [39]:
test['app_date'] = test['app_date'].apply(lambda x: datetime.strptime(x, '%d%b%Y'))

In [40]:
current_date = pd.to_datetime('16APR2021')
# Количество дней, прошедших со дня подачи заявки
train['days'] = (current_date - train.app_date).dt.days
test['days'] = (current_date - test.app_date).dt.days

In [11]:
#train.head()

In [12]:
#test.head()

In [13]:
#sns.countplot(x='default', data=train)

In [44]:
train['default'].value_counts()

0    64427
1     9372
Name: default, dtype: int64

In [45]:
train_0 = train.query('default == 0')
train_1 = train.query('default == 1')
koeff = int(len(train_0)/len(train_1))
for i in range(koeff):
    train = train.append(train_1).reset_index(drop=True)

In [14]:
#sns.countplot(x='default', data=train)

In [47]:
train['default'].value_counts()

1    65604
0    64427
Name: default, dtype: int64

In [48]:
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income', 'days']

In [15]:
#fig, axes = plt.subplots(3, 2, figsize=(25,12))
#for col, i in zip(num_cols, range(6)):
   # sns.distplot(train[col], kde=False, ax=axes.flat[i])

In [50]:
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return lower_bound, upper_bound

In [51]:
num_cols_1 = ['age', 'income','bki_request_cnt','score_bki']
for colname in num_cols_1:
    d = outliers_iqr(train[colname])
    train = train.loc[train[colname].between(d[0], d[1])]

In [52]:
num_cols_log = ['age', 'decline_app_cnt', 'bki_request_cnt', 'days', 'income']

for i in num_cols_log:
    train[i] = np.log(train[i] + 1)

In [53]:
for i in num_cols_log:
    test[i] = np.log(test[i] + 1)

In [16]:
#fig, axes = plt.subplots(3, 2, figsize=(25,12))
#for col, i in zip(num_cols, range(6)):
  #  sns.distplot(train[col], kde=False, ax=axes.flat[i])

In [17]:
#fig, axes = plt.subplots(3, 2, figsize=(25,12))
#for col, i in zip(num_cols, range(6)):
  #  sns.distplot(test[col], kde=False, ax=axes.flat[i])

In [18]:
fig = px.box(train, x="default", y="age")
#fig.show()

NameError: name 'px' is not defined

In [19]:
fig = px.box(train, x="default", y="decline_app_cnt")
#fig.show()

NameError: name 'px' is not defined

In [33]:
fig = px.box(train, x="default", y="bki_request_cnt")
#fig.show()

NameError: name 'px' is not defined

In [32]:
fig = px.box(train, x="default", y="score_bki")
#fig.show()

NameError: name 'px' is not defined

In [31]:
fig = px.box(train, x="default", y="income")
#fig.show()

NameError: name 'px' is not defined

In [30]:
fig = px.box(train, x="default", y="days")
#fig.show()

NameError: name 'px' is not defined

In [20]:
#sns.heatmap(train[num_cols].corr().abs(), vmin=0, vmax=1)

In [21]:
imp_num = Series(f_classif(train[num_cols], train['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
#imp_num.plot(kind = 'barh')

NameError: name 'Series' is not defined

In [64]:
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = [
    'education', 'home_address', 'work_address', 'sna', 'first_time',
    'region_rating'
]

In [22]:
label_encoder = LabelEncoder()

for column in bin_cols:
    train[column] = label_encoder.fit_transform(train[column])
    
# убедимся в преобразовании    
#train.head()

NameError: name 'LabelEncoder' is not defined

In [23]:
for column in bin_cols:
    test[column] = label_encoder.fit_transform(test[column])
    
# убедимся в преобразовании    
#test.head()

NameError: name 'bin_cols' is not defined

In [67]:
train.education.unique()

array(['SCH', 'GRD', 'UGR', 'PGR', 'ACD'], dtype=object)

In [68]:
def edu_code(s):
    if s == "SCH":
        return 0
    elif s == "GRD":
        return 1
    elif s == "UGR":
        return 2
    elif s == "PGR":
        return 3
    elif s == "ACD":
        return 4

In [69]:
train['education'] = train['education'].apply(edu_code)
test['education'] = test['education'].apply(edu_code)

In [24]:
#train.head()

In [25]:
#test.head()

In [26]:
imp_cat = Series(mutual_info_classif(train[bin_cols + cat_cols], train['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
#imp_cat.plot(kind = 'barh')

NameError: name 'Series' is not defined

In [73]:
X_cat = OneHotEncoder(sparse = False).fit_transform(train[cat_cols].values)

In [74]:
Y_cat = OneHotEncoder(sparse = False).fit_transform(test[cat_cols].values)

In [75]:
X_cat.shape

(116236, 26)

In [76]:
X_num = StandardScaler().fit_transform(train[num_cols].values)


In [77]:
Y_num = StandardScaler().fit_transform(test[num_cols].values)

In [78]:
X_num.shape

(116236, 6)

In [79]:
X = np.hstack([X_num, train[bin_cols].values, X_cat])
Y = train['default'].values

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [81]:
from sklearn.model_selection import GridSearchCV

# Добавим типы регуляризации
penalty = ['l1', 'l2']

# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=penalty)

model = LogisticRegression()
model.fit(X_train, y_train)

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, y_train)

# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])


Лучшее Penalty: l2
Лучшее C: 59.94842503189409


In [82]:
model = LogisticRegression(penalty='l2', C=7.742636826811269, max_iter=800)
model.fit(X_train, y_train)

LogisticRegression(C=7.742636826811269, max_iter=800)

In [27]:
probs = model.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
#plt.show()

NameError: name 'model' is not defined

In [84]:
def print_logisitc_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'acc = {acc:.2f} F1-score = {f1:.2f}')


In [85]:
Y_predicted = model.predict(X_test)
print_logisitc_metrics(y_test, Y_predicted)

acc = 0.66 F1-score = 0.67


In [86]:
cm = confusion_matrix(y_test, Y_predicted)
cm

array([[7606, 4074],
       [3737, 7831]], dtype=int64)

In [28]:
sns.set_context(context='paper', font_scale=2, rc=None)
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                cm.flatten()]
labels = [f'{v1}\n{v2}' for v1, v2 in
          zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
#sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

NameError: name 'sns' is not defined

In [88]:
id_test = test['client_id']
test = np.hstack([Y_num, test[bin_cols].values, Y_cat])

In [89]:
model = LogisticRegression(penalty = 'l2', C=2.7825594022071245, max_iter=1000)
model.fit(X, Y)
probs = model.predict_proba(test)
probs = probs[:,1]

In [29]:
my_submission = pd.DataFrame({'client_id': id_test,
                              'default': probs})
my_submission.to_csv('submission.csv', index=False)

#my_submission.head(10)

NameError: name 'pd' is not defined